In [47]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import cross_val_score

### Question 1: Importing Data

In [30]:
rna_seq = pd.read_excel("/Users/mariagarzon/Desktop/Machine-Learning-Course/Assignment_4/RNASeqDat.xlsx")

In [119]:
rna_seq.head()

,RowID,ID,Description,EntrezID,Class,pre_drug.vs.control_ShrunkenLog2FC,pre_drug.vs.control_MLELog2FC,pre_drug.vs.control_pVal,pre_drug.vs.control_pAdj,post_drug.vs.control_ShrunkenLog2FC,...,Raw_Orkambi_021_Base,Raw_Orkambi_021_V2,Raw_Orkambi_022_Base,Raw_Orkambi_022_V2,Raw_Orkambi_024_Base,Raw_Orkambi_024_V2,Raw_Orkambi_025_Base,Raw_Orkambi_025_V2,Raw_Orkambi_026_Base,Raw_Orkambi_026_V2
0,1,A1BG,alpha-1-B glycoprotein,1,protein_coding,-0.050497,-0.066327,0.691693,0.824882,0.082631,...,113,89,31,125,44,78,113,98,38,70
1,2,A1BG-AS1,A1BG antisense RNA 1,503538,lncRNA,0.004178,0.032895,0.971929,0.986309,0.155902,...,40,36,26,87,23,72,74,55,64,53
2,3,A2M-AS1,A2M antisense RNA 1 (head to head),144571,lncRNA,0.356337,0.932311,0.050847,0.179776,-0.359549,...,16,23,40,104,370,199,81,30,228,33
3,4,AAAS,aladin WD repeat nucleoporin,8086,protein_coding,-0.139005,-0.152091,0.285215,0.495514,-0.118870,...,130,71,28,178,62,219,176,106,160,121
4,5,AACS,acetoacetyl-CoA synthetase,65985,protein_coding,0.138032,0.143003,0.088856,0.251817,0.075925,...,192,157,71,228,130,185,198,114,131,157


Filter the dataframe to only contain the 60 columns that have the raw data

In [88]:
#Get column names
rna_seq.info()
for col in rna_seq.columns:
    print(col)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15570 entries, 0 to 15569
Columns: 140 entries, RowID to Raw_Orkambi_026_V2
dtypes: float64(75), int64(62), object(3)
memory usage: 16.6+ MB
RowID
ID
Description
EntrezID
Class
pre_drug.vs.control_ShrunkenLog2FC
pre_drug.vs.control_MLELog2FC
pre_drug.vs.control_pVal
pre_drug.vs.control_pAdj
post_drug.vs.control_ShrunkenLog2FC
post_drug.vs.control_MLELog2FC
post_drug.vs.control_pVal
post_drug.vs.control_pAdj
post_drug.vs.pre_drug_ShrunkenLog2FC
post_drug.vs.pre_drug_MLELog2FC
post_drug.vs.pre_drug_pVal
post_drug.vs.pre_drug_pAdj
control_Mean
pre-drug_Mean
post-drug_Mean
Norm_10_HC_Auto_066_237
Norm_11_Orkambi_006_Base
Norm_12_Orkambi_007_V2
Norm_13_HC_Auto_068_239
Norm_14_Orkambi_007_Base
Norm_15_Orkambi_009_V2
Norm_16_HC_Auto_072_243
Norm_17_Orkambi_009_Base
Norm_18_Orkambi_010_V2
Norm_19_HC_Auto_074_245
Norm_1_Orkambi_001_Base
Norm_20_Orkambi_010_Base
Norm_21_HC_Immune_004
Norm_22_Orkambi_012_V2
Norm_23_HC_Auto_076_247
Norm_24_Orkambi_

In [154]:
#Slice Dataframe to only include raw data
raw_data = rna_seq.filter(like='Raw')

In [155]:
raw_data.head()

,Raw_10_HC_Auto_066_237,Raw_11_Orkambi_006_Base,Raw_12_Orkambi_007_V2,Raw_13_HC_Auto_068_239,Raw_14_Orkambi_007_Base,Raw_15_Orkambi_009_V2,Raw_16_HC_Auto_072_243,Raw_17_Orkambi_009_Base,Raw_18_Orkambi_010_V2,Raw_19_HC_Auto_074_245,...,Raw_Orkambi_021_Base,Raw_Orkambi_021_V2,Raw_Orkambi_022_Base,Raw_Orkambi_022_V2,Raw_Orkambi_024_Base,Raw_Orkambi_024_V2,Raw_Orkambi_025_Base,Raw_Orkambi_025_V2,Raw_Orkambi_026_Base,Raw_Orkambi_026_V2
0,81,38,112,46,64,196,125,140,72,104,...,113,89,31,125,44,78,113,98,38,70
1,49,31,84,26,38,86,80,76,38,56,...,40,36,26,87,23,72,74,55,64,53
2,74,108,62,47,41,31,105,45,63,202,...,16,23,40,104,370,199,81,30,228,33
3,161,86,197,92,116,143,243,136,151,149,...,130,71,28,178,62,219,176,106,160,121
4,199,128,198,117,162,263,218,243,201,152,...,192,157,71,228,130,185,198,114,131,157


In [156]:
#Add gene id names to the raw data
gene_ids=rna_seq["ID"]
raw_data.insert(0, "Gene_ID", gene_ids)
raw_data.head()

,Gene_ID,Raw_10_HC_Auto_066_237,Raw_11_Orkambi_006_Base,Raw_12_Orkambi_007_V2,Raw_13_HC_Auto_068_239,Raw_14_Orkambi_007_Base,Raw_15_Orkambi_009_V2,Raw_16_HC_Auto_072_243,Raw_17_Orkambi_009_Base,Raw_18_Orkambi_010_V2,...,Raw_Orkambi_021_Base,Raw_Orkambi_021_V2,Raw_Orkambi_022_Base,Raw_Orkambi_022_V2,Raw_Orkambi_024_Base,Raw_Orkambi_024_V2,Raw_Orkambi_025_Base,Raw_Orkambi_025_V2,Raw_Orkambi_026_Base,Raw_Orkambi_026_V2
0,A1BG,81,38,112,46,64,196,125,140,72,...,113,89,31,125,44,78,113,98,38,70
1,A1BG-AS1,49,31,84,26,38,86,80,76,38,...,40,36,26,87,23,72,74,55,64,53
2,A2M-AS1,74,108,62,47,41,31,105,45,63,...,16,23,40,104,370,199,81,30,228,33
3,AAAS,161,86,197,92,116,143,243,136,151,...,130,71,28,178,62,219,176,106,160,121
4,AACS,199,128,198,117,162,263,218,243,201,...,192,157,71,228,130,185,198,114,131,157


### Question 2: Normalization

Per million scaling to control variation in sequencing depth. 
1. Compue relative abundance: for each gene g, compute proportion of reads/counts assigned to g relative to the total number of reads in that sample. 
2. Scale to a fixed value: multiply relative abundance by a scaling factor (1 million) to make numbers more manageable and comparable across samples. 

Subheading a:

In [157]:
raw_cols_sums = raw_data[1:61].sum(axis = 0, skipna = True)  # Calculate the sum of each column
print(raw_cols_sums)
# Create a copy of the DataFrame for normalization
norm_cpm_df = raw_data.copy()

Gene_ID                    A1BG-AS1A2M-AS1AAASAACSAAED1AAGABAAK1AAMDCAAMP...
Raw_10_HC_Auto_066_237                                                 49554
Raw_11_Orkambi_006_Base                                                36371
Raw_12_Orkambi_007_V2                                                  59219
Raw_13_HC_Auto_068_239                                                 35697
                                                 ...                        
Raw_Orkambi_024_V2                                                     52219
Raw_Orkambi_025_Base                                                   57331
Raw_Orkambi_025_V2                                                     53249
Raw_Orkambi_026_Base                                                   31577
Raw_Orkambi_026_V2                                                     45619
Length: 61, dtype: object


In [158]:
raw_cols_sums.iat[0]=0

In [159]:
#Insert total row sum at the end of columns
raw_data.loc[len(raw_data.index)] = raw_cols_sums
raw_data.tail()

/var/folders/g8/064nc11n7yl2sz1wtj_388d40000gn/T/ipykernel_12351/4151776668.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data.loc[len(raw_data.index)] = raw_cols_sums


,Gene_ID,Raw_10_HC_Auto_066_237,Raw_11_Orkambi_006_Base,Raw_12_Orkambi_007_V2,Raw_13_HC_Auto_068_239,Raw_14_Orkambi_007_Base,Raw_15_Orkambi_009_V2,Raw_16_HC_Auto_072_243,Raw_17_Orkambi_009_Base,Raw_18_Orkambi_010_V2,...,Raw_Orkambi_021_Base,Raw_Orkambi_021_V2,Raw_Orkambi_022_Base,Raw_Orkambi_022_V2,Raw_Orkambi_024_Base,Raw_Orkambi_024_V2,Raw_Orkambi_025_Base,Raw_Orkambi_025_V2,Raw_Orkambi_026_Base,Raw_Orkambi_026_V2
15566,ZYG11B,2591,2248,2374,2005,2406,3574,4167,4939,3513,...,3103,3112,1895,4289,2191,2274,2857,3537,1056,2432
15567,ZYX,826,1097,836,610,1431,1166,1411,3067,2651,...,1592,1118,950,2321,1160,1215,1144,872,2198,876
15568,ZZEF1,2552,2668,2583,2093,2486,2645,3005,4363,3036,...,2413,2023,960,3538,2939,3225,2370,2050,4532,2432
15569,ZZZ3,1743,1095,2415,1046,1343,2157,2279,2001,1690,...,1579,1734,654,2152,1153,1772,1768,1428,946,1362
15570,0,49554,36371,59219,35697,43462,64669,64171,67510,60646,...,45774,48848,18034,60077,33020,52219,57331,53249,31577,45619


In [160]:
norm_cpm_df = pd.DataFrame(index=raw_data.index, columns=raw_data.columns)

for i in range(len(raw_data)):
    for j in range(1, len(raw_data.columns)):
        norm_cpm_df.iat[i,j] = (int(raw_data.iat[i,j]) * 1000000) / int(raw_cols_sums[j])

In [161]:
norm_cpm_df['Gene_ID'] = rna_seq['ID']

In [162]:
norm_cpm_df.head()

,Gene_ID,Raw_10_HC_Auto_066_237,Raw_11_Orkambi_006_Base,Raw_12_Orkambi_007_V2,Raw_13_HC_Auto_068_239,Raw_14_Orkambi_007_Base,Raw_15_Orkambi_009_V2,Raw_16_HC_Auto_072_243,Raw_17_Orkambi_009_Base,Raw_18_Orkambi_010_V2,...,Raw_Orkambi_021_Base,Raw_Orkambi_021_V2,Raw_Orkambi_022_Base,Raw_Orkambi_022_V2,Raw_Orkambi_024_Base,Raw_Orkambi_024_V2,Raw_Orkambi_025_Base,Raw_Orkambi_025_V2,Raw_Orkambi_026_Base,Raw_Orkambi_026_V2
0,A1BG,1634.580458,1044.78843,1891.284892,1288.623694,1472.550734,3030.818476,1947.9204,2073.766849,1187.217624,...,2468.650326,1821.978382,1718.975269,2080.663149,1332.525742,1493.709186,1971.010448,1840.410149,1203.407543,1534.448366
1,A1BG-AS1,988.820277,852.327404,1418.463669,728.352523,874.326998,1329.848923,1246.669056,1125.759147,626.587079,...,873.858522,736.98002,1441.721193,1448.141552,696.547547,1378.80848,1290.750205,1032.883247,2026.791652,1161.79662
2,A2M-AS1,1493.320418,2969.398697,1046.961279,1316.637252,943.352814,479.364147,1636.253136,666.567916,1038.815421,...,349.543409,470.848346,2218.032605,1731.11174,11205.330103,3810.873437,1412.848197,563.390862,7220.445261,723.382801
3,AAAS,3248.98091,2364.521184,3326.635033,2577.247388,2668.998205,2211.260418,3786.757258,2014.516368,2489.859183,...,2840.040197,1453.488372,1552.622824,2962.864324,1877.649909,4193.875792,3069.892379,1990.647712,5066.97913,2652.403604
4,AACS,4015.821124,3519.287344,3343.521505,3277.586352,3727.394045,4066.863567,3397.173178,3599.466746,3314.315866,...,4194.520907,3214.051752,3937.007874,3795.129584,3937.007874,3542.771788,3453.628927,2140.885275,4148.589163,3441.548478


### Question 3: Top 10 genes

#### Subheading a:

The columns of the dataframe are labeled as such:
- any column name that has the term HC is a Health Control
- any column with Base is a patient with CF but no Treatment
- any column with V2 is a CF patient with treatment

In [163]:
# List of top 10 genes
top10_genes = ['LOC105372578', 'MCEMP1', 'MMP9', 'SOCS3', 'ANXA3', 'G0S2', 'IL1R2', 'PFKFB3', 'OSM', 'SEMA6B']

# Setting 'Gene_ID' as the index of norm_cpm_df
norm_cpm_df.set_index('Gene_ID', inplace=True)

# Filtering norm_cpm_df to contain only the genes in top10_genes
norm_cpm_df_top10 = norm_cpm_df.loc[top10_genes]

In [164]:
norm_cpm_df_top10.head()

,Raw_10_HC_Auto_066_237,Raw_11_Orkambi_006_Base,Raw_12_Orkambi_007_V2,Raw_13_HC_Auto_068_239,Raw_14_Orkambi_007_Base,Raw_15_Orkambi_009_V2,Raw_16_HC_Auto_072_243,Raw_17_Orkambi_009_Base,Raw_18_Orkambi_010_V2,Raw_19_HC_Auto_074_245,...,Raw_Orkambi_021_Base,Raw_Orkambi_021_V2,Raw_Orkambi_022_Base,Raw_Orkambi_022_V2,Raw_Orkambi_024_Base,Raw_Orkambi_024_V2,Raw_Orkambi_025_Base,Raw_Orkambi_025_V2,Raw_Orkambi_026_Base,Raw_Orkambi_026_V2
Gene_ID,,,,,,,,,,,,,,,,,,,,,
LOC105372578,1594.220446,3794.231668,1182.053057,2549.233829,17210.436703,6401.830862,1683.003226,47948.452081,46103.617716,1359.908206,...,9415.825578,13183.753685,51846.512144,57492.884132,8843.125379,2719.316724,6383.980743,12188.02231,1045.064446,7365.352156
MCEMP1,2683.940752,2447.004482,1300.258363,2549.233829,17325.479729,5010.128501,2290.754391,34587.468523,26959.733536,1508.648166,...,15489.142308,9314.608582,5766.884773,13732.376783,2604.482132,1876.711542,6907.257854,7943.811151,1805.111315,3003.134659
MMP9,22056.746176,34533.007066,39345.480336,31627.307617,219064.930284,155221.203359,26320.300447,377025.625833,158163.770076,11028.00561,...,157600.384498,81252.047167,82067.206388,241773.057909,40642.03513,24646.201574,80305.593832,137035.437285,35500.522532,57717.179245
SOCS3,30572.70856,66124.109868,19942.923724,44653.612348,203258.018499,75476.658059,44708.669025,206221.300548,370428.387693,20398.623093,...,167453.139337,149217.982312,341632.471997,239126.454384,54603.270745,35772.42,116708.238126,185186.576274,17449.40938,73587.759486
ANXA3,25285.54708,35605.289929,9895.472737,44569.571673,177327.320418,54570.195921,39940.159885,339105.317731,216749.661973,17105.095406,...,113994.844235,131898.951851,161029.167129,168816.685254,73743.185948,36059.671767,91660.707122,177280.324513,9532.254489,62298.603652


#### Subheading b:

Split norm_cpm_df_top10 into 2 different dataframes:

- one with just healthy values = hc_top10
- one with just CF (Base) values = cf_top10

In [165]:
hc_top10 = norm_cpm_df_top10.filter(like='HC')
cf_top10 = norm_cpm_df_top10.filter(like='Base')

#### Subheading c:

Transpose both data frames

In [166]:
hc_top10_t = hc_top10.T
cf_top10_t = cf_top10.T

In [167]:
hc_top10_t['Y'] = 0  # Adding label 0 for healthy control
cf_top10_t['Y'] = 1  # Adding label 1 for CF Base

### Subheading d:

In [171]:
# Concatenating the DataFrames vertically
hc_cf_top10 = pd.concat([hc_top10_t, cf_top10_t], axis=0)
# Resetting the index
hc_cf_top10.reset_index(drop=True, inplace=True)

In [172]:
hc_cf_top10.head()

Gene_ID,LOC105372578,MCEMP1,MMP9,SOCS3,ANXA3,G0S2,IL1R2,PFKFB3,OSM,SEMA6B,Y
0,1594.220446,2683.940752,22056.746176,30572.70856,25285.54708,565.040158,25265.367074,18141.82508,3148.080881,565.040158,0
1,2549.233829,2549.233829,31627.307617,44653.612348,44569.571673,784.37964,44681.625907,33560.243158,4426.142253,532.257613,0
2,1683.003226,2290.754391,26320.300447,44708.669025,39940.159885,997.335245,41140.078852,31213.476492,4410.091786,498.667622,0
3,1359.908206,1508.648166,11028.00561,20398.623093,17105.095406,956.185457,29726.743445,19931.154647,2422.336492,488.717012,0
4,3302.721375,2948.858371,25983.654899,51765.102368,46019.0412,909.93344,45917.937484,26270.115427,5156.289494,556.070436,0


### Question 4

Subheading a:

C: The C parameter trades off correct classification of training examples against maximization of the decision function’s margin. 
Lower C encourages a larger margin, simpler decision function, at the cost of training accuracy (potential to underfit the data)
Higher C encourages a smaller margin, complex decision function, potentially overfitting the data.

Kernel: Specifies the kernel type to be used in the algorithm. It must be one of ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’, or a callable. Here we're using 'linear' and 'rbf'. 

cross_val_score: This function is used to evaluate a score by cross-validation. In this case, we use 5-fold cross-validation (cv=5).

In [173]:
X = hc_cf_top10.drop(columns=['Y'])
Y = hc_cf_top10['Y']

In [175]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import numpy as np

In [176]:
# Values for C and kernels to try
C_values = [0.5, 1, 2]  # Less than 1, equal to 1, and more than 1
kernels = ['linear', 'rbf']  # Linear and radial (rbf) kernels

# Placeholder for results
results = {}

# Loop through kernels and C values, perform 5-fold cross-validation
for kernel in kernels:
    for C_value in C_values:
        svc = SVC(C=C_value, kernel=kernel, random_state=1)
        scores = cross_val_score(svc, X, Y, cv=5)  # 5-fold cross-validation
        results[f'Kernel={kernel}, C={C_value}'] = np.mean(scores)

# Display results
for config, score in results.items():
    print(f"{config}: Accuracy: {score*100:.2f}%")

Kernel=linear, C=0.5: Accuracy: 80.00%
Kernel=linear, C=1: Accuracy: 80.00%
Kernel=linear, C=2: Accuracy: 80.00%
Kernel=rbf, C=0.5: Accuracy: 82.50%
Kernel=rbf, C=1: Accuracy: 82.50%
Kernel=rbf, C=2: Accuracy: 85.00%


The best combination of kernel and C parameter for your Support Vector Machine (SVM) based on the 5-fold cross-validation accuracy is:

Kernel: rbf (Radial basis function)
C: 2
Accuracy: 85.00%

### Subheading c: Explanation of Score Calculation

Step 1: Splitting the Data
The dataset (hc_cf_top10) is split into 5 (or "k") subsets (folds) of roughly equal size.
This is done in a stratified manner if possible (keeping the distribution of the output variable Y consistent across all folds) to ensure each fold is a good representative of the overall dataset.

Step 2: Model Training and Evaluation
The model is trained k times (5 times in this case).
In each iteration, one fold is used as the validation set, and the remaining k-1 folds are used to train the model.
After being trained, the model is evaluated using the reserved validation fold.
The accuracy of the model for each iteration (each fold used as validation once) is recorded.

Step 3: Averaging the Results
The accuracies from all k iterations are averaged to produce a single score.
This average score is the one reported in your results, e.g., 85.00% for Kernel=rbf and C=2.

Why Cross-Validation?
Reduces Bias: By using different subsets of the data for training and validation, it ensures that the results are not dependent on a particular train-test split, providing a more robust estimate of the model's performance.
Utilizes Data Efficiently: Especially in scenarios where data is limited, using k-fold cross-validation ensures that every data point is used for validation exactly once.
Provides a More Robust Performance Estimate: The average performance metric (e.g., accuracy) across all folds provides a more stable and reliable estimate of model performance than a single train/test split.